In [1]:
import pandas as pd
data = pd.read_csv("/kaggle/input/petfinder-pawpularity-score/train.csv", sep=',')
data

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9907,ffbfa0383c34dc513c95560d6e1fdb57,0,0,0,1,0,0,0,0,0,0,0,1,15
9908,ffcc8532d76436fc79e50eb2e5238e45,0,1,1,1,0,0,0,0,0,0,0,0,70
9909,ffdf2e8673a1da6fb80342fa3b119a20,0,1,1,1,0,0,0,0,1,1,0,0,20
9910,fff19e2ce11718548fa1c5d039a5192a,0,1,1,1,0,0,0,0,1,0,0,0,20


Data augmentation basique : on ajoute la moyenne RGB de l'image sur 3 colonnes

In [4]:
dataset_dir = '/kaggle/input/petfinder-pawpularity-score/train'

import cv2
import os

for img_id in data['Id']:
    img = cv2.imread(os.path.join(dataset_dir, img_id + '.jpg'))
    h, w, c = img.shape
    l = 0
    b_avg = 0; g_avg = 0; r_avg = 0
    for i in range(h):
        for j in range(w):
            if (img[i, j, 0] != 0 or img[i, j, 1] != 0 or img[i, j, 2] != 0):
                l += 1
                b_avg = b_avg + img[i, j, 0]
                g_avg = g_avg + img[i, j, 1]
                r_avg = r_avg + img[i, j, 2]
    data.loc[data['Id'] == img_id, ['r']] = r_avg / (l * 255)
    data.loc[data['Id'] == img_id, ['g']] = g_avg / (l * 255)
    data.loc[data['Id'] == img_id, ['b']] = b_avg / (l * 255)

data

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,r,g,b
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63,0.595822,0.586457,0.565182
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42,0.414034,0.363036,0.332507
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28,0.386519,0.495081,0.476460
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15,0.626640,0.613553,0.596168
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72,0.529374,0.500299,0.483081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9907,ffbfa0383c34dc513c95560d6e1fdb57,0,0,0,1,0,0,0,0,0,0,0,1,15,NaN,NaN,NaN
9908,ffcc8532d76436fc79e50eb2e5238e45,0,1,1,1,0,0,0,0,0,0,0,0,70,NaN,NaN,NaN
9909,ffdf2e8673a1da6fb80342fa3b119a20,0,1,1,1,0,0,0,0,1,1,0,0,20,NaN,NaN,NaN
9910,fff19e2ce11718548fa1c5d039a5192a,0,1,1,1,0,0,0,0,1,0,0,0,20,NaN,NaN,NaN


In [ ]:
import pickle
data.to_pickle("data_rgb.pkl")

In [ ]:
X = data.drop(columns=['Id', 'Pawpularity'])
y = data['Pawpularity']

from sklearn.model_selection import train_test_split, GridSearchCV, ParameterGrid
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                    random_state=1)

from catboost import CatBoostRegressor
import time

#parameters = {
#    'learning_rate': [ 0.01, 0.05, 0.1, 0.2 ],
#    'depth'    : [ 1, 3, 5 ],
#    'n_estimators' : [ 50, 100, 500 ]
#}
#tps1 = time.time()
#print("Nombre de combinaisons de paramètres étudiées :", len(ParameterGrid(parameters)))
#grid_search_cv = GridSearchCV(estimator = CatBoostRegressor(random_state=1), param_grid = parameters, cv = 3, n_jobs=-1)
#grid_search_cv.fit(X_train, y_train)
#print("best estimator :", grid_search_cv.best_estimator_)
#print("best score :", grid_search_cv.best_score_)
#print("best params :", grid_search_cv.best_params_)
#tps2 = time.time()
#print("\nTemps de traitement total :", tps2 - tps1, "secondes")

# Nombre de combinaisons de paramètres étudiées : 36
# best estimator : <catboost.core.CatBoostRegressor object at 0x7fd4cc759590>
# best score : -0.0014066586341349903
# best params : {'depth': 5, 'learning_rate': 0.01, 'n_estimators': 50}
# Temps de traitement total : 20.465460300445557 secondes

In [ ]:
start = time.time()

cb = CatBoostRegressor(depth=5,
                       learning_rate=0.01,
                       n_estimators=50,
                       loss_function='RMSE',
                       task_type='CPU',
                       verbose=False)

cb.fit(X_train, y_train)
y_pred = cb.predict(X_test)

In [ ]:
import numpy as np
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse

cb_rmse = np.sqrt(mse(y_test, y_pred))
print("RMSE for CatBoost: ", np.mean(cb_rmse))

end = time.time()
diff = end - start
print('Execution time for CatBoost (in Seconds):', diff)

print("r2_score : {:.2f}".format(r2_score(y_test, y_pred)))

In [ ]:
sub_df = pd.read_csv("/kaggle/input/petfinder-pawpularity-score/test.csv", sep=',')
X_sub = sub_df.drop(columns=['Id'])
print("shape X_sub: ", X_sub.shape)
y_sub = cb.predict(X_sub)
y_sub = [round(x) for x in y_sub]
sub_df['Pawpularity'] = y_sub
sub_df = sub_df.drop(columns=['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'])
print(sub_df)
sub_df.to_csv('submission.csv', index=False, sep=',')